In [ ]:
import cv2
import os
import numpy as np
import shutil
from pathlib import Path

PROJECT_ROOT = Path.cwd()
OUTPUT_ROOT = Path(os.getenv("VSS_OUTPUT_ROOT", PROJECT_ROOT / "outputs"))
DATA_ROOT = Path(os.getenv("VSS_DATA_ROOT", PROJECT_ROOT / "data"))
SPLIT_ROOT = Path(os.getenv("VSS_SPLIT_ROOT", DATA_ROOT / "splits"))

empty_dest = True

def show_img(img, state):
    # Display
    cv2.imshow(f"{state}", img)
    #cv2.waitKey(wait)
    #cv2.destroyAllWindows()

# Define the path to the folder containing the images
input_folder = OUTPUT_ROOT / "FP_HD_groundtruth_512"
output_folder = OUTPUT_ROOT / "FP_HD_area_BonW_512"

# Make sure the output folder exists, if not create it
if not output_folder.exists():
    output_folder.mkdir(parents=True, exist_ok=True)

# Optionally empty the destination folders
if empty_dest:
    for folder in [output_folder]:
        for filename in os.listdir(folder):
            file_path = os.path.join(folder, filename)
            try:
                if os.path.isfile(file_path) or os.path.islink(file_path):
                    os.unlink(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
            except Exception as e:
                print(f"Failed to delete {file_path}. Reason: {e}")

# Loop through all PNG images in the input folder
for file_path in input_folder.glob("*.png"):
    # Read the image
    image = cv2.imread(str(file_path), cv2.IMREAD_UNCHANGED)

    # Convert to grayscale
    if len(image.shape) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply binary thresholding
    _, binary = cv2.threshold(image, 10, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a white image with the same size as the original image
    filled_image = np.ones_like(image) * 255

    # Fill the contours with black color (0)
    cv2.drawContours(filled_image, contours, -1, 0, thickness=cv2.FILLED)

    # Save the modified image
    output_file_path = output_folder / file_path.name
    cv2.imwrite(str(output_file_path), filled_image)

print("Conversion complete. Check the output folder for results.")



In [ ]:
from PIL import Image
import os
import shutil
from pathlib import Path

empty_dest = True

PATH = OUTPUT_ROOT

# Paths to your directories
input_folder_path = PATH / "FP_HD_groundtruth_512"
real_folder_path = [PATH / "FP_HD_area_BonW_512"]

for path in real_folder_path:
    # Extract part of the name from the path
    name_part = path.name.replace("FP_outline_", "").replace("FP_outline_filled_", "")
    # Construct the output folder path
    output_folder_path = PATH / "FP_HD_train_512"

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    # Get lists of png files from both folders
    input_images = sorted([f for f in os.listdir(input_folder_path) if f.endswith('.png')])
    real_images = sorted([f for f in os.listdir(path) if f.endswith('.png')])

    # Check if both lists have same length
    if len(input_images) != len(real_images):
        raise ValueError("The number of images in both folders is not the same.")

    # Optionally empty the destination folders
    if empty_dest:
        for folder in [output_folder_path]:
            for filename in os.listdir(folder):
                file_path = os.path.join(folder, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"Failed to delete {file_path}. Reason: {e}")

    # Assuming the images are named the same in both folders
    # Pair and process images
    # Get dimensions
    w, h = 512, 512
    wiviu = 4000
    counter = 0
    for i, (input_filename, real_filename) in enumerate(zip(input_images, real_images)):
        if counter < wiviu:
            counter += 1
            # Construct full paths
            input_image_path = os.path.join(input_folder_path, input_filename)
            real_image_path = os.path.join(path, real_filename)

            output_filename = f"{str(i+1)}.png"
            output_image_path = os.path.join(output_folder_path, output_filename)

            # Open the images
            input_image = Image.open(input_image_path).resize((w, h))
            real_image = Image.open(real_image_path).resize((w, h))

            # Create a new image with double the width
            combined_image = Image.new('RGB', (2 * w, h))

            # Paste the images side by side in the new image
            combined_image.paste(input_image, (0, 0))
            combined_image.paste(real_image, (w, 0))

            # Save the new image
            combined_image.save(output_image_path)

    print("Images combined and saved in:", output_folder_path)



In [ ]:
import os
import shutil
from pathlib import Path
from random import shuffle


def split_dataset(source_folder, dest_folder_1, dest_folder_2, split_ratio=0.5, empty_dest=True):
    """
    Splits images from the source folder into two destination folders with given split ratio.

    :param source_folder: Path to the source folder containing images
    :param dest_folder_1: Path to the first destination folder
    :param dest_folder_2: Path to the second destination folder
    :param split_ratio: Ratio of images to be moved to the first destination folder. Default is 0.5.
    """
    # Ensure the destination folders exist, if not create them
    os.makedirs(dest_folder_1, exist_ok=True)
    os.makedirs(dest_folder_2, exist_ok=True)

    # Optionally empty the destination folders
    if empty_dest:
        for folder in [dest_folder_1, dest_folder_2]:
            for filename in os.listdir(folder):
                file_path = os.path.join(folder, filename)
                try:
                    if os.path.isfile(file_path) or os.path.islink(file_path):
                        os.unlink(file_path)
                    elif os.path.isdir(file_path):
                        shutil.rmtree(file_path)
                except Exception as e:
                    print(f"Failed to delete {file_path}. Reason: {e}")

    # List all files in the source folder
    files = [f for f in os.listdir(source_folder) if os.path.isfile(os.path.join(source_folder, f))]
    print(len(files))

    # Calculate the split index
    split_idx = int(len(files) * split_ratio)

    # Split files into two sets
    files_1 = files[:split_idx]
    files_2 = files[split_idx:]
    print(f"Moving {len(files_1)} into test and {len(files_2)} into train")

    # Copy files to destination folders
    for i, filename in enumerate(files_1):
        src_path = os.path.join(source_folder, filename)
        dest_path = os.path.join(dest_folder_1, f"{i+1}.png")
        shutil.copy(src_path, dest_path)

    for i, filename in enumerate(files_2):
        src_path = os.path.join(source_folder, filename)
        dest_path = os.path.join(dest_folder_2, f"{i+1}.png")
        shutil.copy(src_path, dest_path)


PATH = OUTPUT_ROOT
source_sets = [PATH / "FP_combined"]

for dataset in source_sets:
    source_folder = str(dataset)
    print(source_folder)
    identifier = dataset.name.replace("FP_combine_", "")
    split_dataset(
        source_folder,
        str(SPLIT_ROOT / f"test{identifier}"),
        str(SPLIT_ROOT / f"train{identifier}"),
        split_ratio=0.5,
        empty_dest=True,
    )

